# Plot paper figures
Download data at: http://doi.org/10.4121/uuid:274bdd06-14a5-45c3-bc86-87d400082e34

In [ ]:
import holoviews as hv
import matplotlib.pylab as plt
import matplotlib.patches as patches
import numpy as np
from shortjunction import constants, SimpleNamespace
hv.notebook_extension()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {
          'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 1000
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

In [ ]:
%output fig='svg' size=100 dpi=600
%opts Path [aspect='square'] (lw=0.5) Image [aspect='square' colorbar=True] (cmap='inferno_r')
img = hv.Store.load(open('data/decay_length_2d.h', 'rb'))
img

In [ ]:
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125,0.2,(0.95-0.125),(0.95-0.2)])
B_min, mu_min, B_max, mu_max = img.bounds.lbrt()
extent = [B_min, B_max / constants.unit_B, mu_min / constants.delta_2d, mu_max / constants.delta_2d]
cax = ax.imshow(img.data / 100, vmin=0, vmax=30, extent=extent, cmap='inferno_r', aspect='auto')
ax.set_yticks([2, 4, 6, 8])
cax.set_clim(0, 30)
ax.set_ylabel('Chemical potential $\mu$ [$\delta$]')
ax.set_xlabel('Magnetic field $B$ [$2\delta/g\mu_B$]')
cbar_ticks = [0, 5, 10, 15, 20, 25, 30, 35]
cbar = fig.colorbar(cax, ticks=cbar_ticks, extend='max', label=r'$\xi$ [W]')
cbar.ax.set_yticklabels(cbar_ticks)
fig.savefig('decay_length_2d.pdf', format='pdf', dpi=1000)
plt.show()

# 3D phase diagram with bandgaps

In [ ]:
phase_diagram_3d_no_orbital_100x100 = hv.Store.load(open('data/phase_diagram_3d_no_orbital_100x100.h', 'rb'))
phase_diagram_3d_no_orbital_120x120 = hv.Store.load(open('data/phase_diagram_3d_no_orbital_120x120.h', 'rb'))
phase_diagram_3d_with_orbital_100x100 = hv.Store.load(open('data/phase_diagram_3d_with_orbital_100x100.h', 'rb'))
phase_diagram_3d_with_orbital_120x120 = hv.Store.load(open('data/phase_diagram_3d_with_orbital_120x120.h', 'rb'))
B_min, mu_min, B_max, mu_max = phase_diagram_3d_no_orbital_100x100.bounds.lbrt()
extent = [B_min, B_max, mu_min, mu_max]

(phase_diagram_3d_no_orbital_100x100 +
 phase_diagram_3d_no_orbital_120x120 +
 phase_diagram_3d_with_orbital_100x100 +
 phase_diagram_3d_with_orbital_120x120)

In [ ]:
fig = plt.figure()
plt.rcParams['figure.figsize'] = (fig_width, fig_height+0.5)
plt.rcParams['axes.titlesize'] = 9
fig, axs = plt.subplots(2, 2, sharey=True, sharex=True)
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.33, wspace=0.15)
kwargs = dict(extent=extent, cmap='inferno_r', aspect='auto', vmin=0, vmax=1)
axs[0][0].imshow(phase_diagram_3d_no_orbital_100x100.data, **kwargs)
axs[0][1].imshow(phase_diagram_3d_no_orbital_120x120.data, **kwargs)
axs[1][0].imshow(phase_diagram_3d_with_orbital_100x100.data, **kwargs)
im = axs[1][1].imshow(phase_diagram_3d_with_orbital_120x120.data, **kwargs)
im.set_clim(0, 1)
axs[1][0].set_xlabel('$B$ [T]')
axs[1][1].set_xlabel('$B$ [T]')
axs[0][0].set_ylabel('$\mu$ [meV]')
axs[1][0].set_ylabel('$\mu$ [meV]')
axs[0][0].set_yticks(np.arange(0, 16, 5))


axs[0][1].add_patch(patches.Rectangle((0.01 * extent[1], 
                                      0.02 * (extent[-1] - extent[2])), 
                                     extent[1] / 1.2**2, 
                                     extent[-1] / 1.2**2, fill=False))

axs[1][1].add_patch(patches.Rectangle((0.01 * extent[1], 
                                      0.02 * (extent[-1] - extent[2])), 
                                     extent[1] / 1.2**2, 
                                     extent[-1] / 1.2**2, fill=False))

cax = fig.add_axes([0.83, 0.2, 0.03, 0.7])
cb = fig.colorbar(im, cax=cax, label=r'$\Delta_\mathrm{spec}/\Delta$')
cb.set_ticks([0.0, 0.2, 0.4, 0.6, 0.8, 1])

labels = ['(a)', '(b)', '(c)', '(d)'] 
titles = [r'no orb, $W=100$ nm', r'no orb, $W=120$ nm', 
          r'orb, $W=100$ nm', r'orb, $W=120$ nm']

for ax, label, title in zip(axs.reshape(-1), labels, titles):
    ax.text(.02, 12.5, label, color='white')
    ax.set_title(title)

fig.savefig('phase_diagrams_3d.pdf', format='pdf', dpi=600)
plt.show()